In [1]:
# Import necessary libraries
import xarray as xr
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import os
import glob

In [2]:
import sys
# Get the current directory of the notebook (replace this with your notebook's directory if not running interactively)
notebook_dir = os.getcwd()

# Construct the path to the module
module_path = os.path.abspath(os.path.join(notebook_dir, "../src/"))

# Add the module's directory to sys.path
if module_path not in sys.path:
    sys.path.append(module_path)

In [3]:
# Base directory for your datasets
# base_dir = "/mnt/d/tom_data/SWOT"
base_dir = "/home/yugao/SWOT_L2/data/external"

In [4]:
# List files and directories in your data folder
for name in os.listdir(base_dir):
    print(name)

.ipynb_checkpoints
.gitkeep
L2_LR_unsmoothed
L2_LR_SSH
L2_LR_windwave
L2_LR_basic
L2_LR_expert


In [5]:
subdir_name = "L2_LR_windwave"
# subdir_name = "L2_LR_expert"
file_pattern="*.nc"

In [6]:
path_pattern = os.path.join(base_dir, subdir_name, file_pattern)

In [16]:
# Define the bounding box for Martha's Vineyard with the given bounds
# center point of Matha's Vinyard: 41°24′N 70°37′W

lat_min, lat_max = 40.25, 41.75 # Latitude bounds 41N =1.5
lon_min, lon_max = -71, -69.5 # Longitude bounds  = 1.5

lolabox = [lon_min, lon_max, lat_min, lat_max]

In [17]:
file_path = glob.glob(path_pattern)

In [18]:
img_dir = f"/home/yugao/SWOT_L2/img/{subdir_name}"
processed_dir = f"/home/yugao/SWOT_L2/data/processed/{subdir_name}"

## Initial exploration: crop and plot the wind and wave data

In [19]:
# ds_sub = crop_and_plot_wind_data(file_path[:], lolabox, img_dir, processed_dir)
# crop_and_plot_wind_data(file_path[:], lolabox, processed_dir)

## Visualize subset

In [20]:
# Define the base directory where images will be saved
img_dir = f"/home/yugao/SWOT_L2/img/{subdir_name}"

# Ensure the target directory exists, create if it doesn't
os.makedirs(img_dir, exist_ok=True)

nc_files = glob.glob(processed_dir + '/*.nc')
nc_files

['/home/yugao/SWOT_L2/data/processed/L2_LR_windwave/SWOT_L2_LR_SSH_2024-03-06_225821.259004.nc',
 '/home/yugao/SWOT_L2/data/processed/L2_LR_windwave/SWOT_L2_LR_SSH_2024-02-14_132300.150082.nc',
 '/home/yugao/SWOT_L2/data/processed/L2_LR_windwave/SWOT_L2_LR_SSH_2024-02-13_132222.628954.nc',
 '/home/yugao/SWOT_L2/data/processed/L2_LR_windwave/SWOT_L2_LR_SSH_2024-01-13_181502.310175.nc',
 '/home/yugao/SWOT_L2/data/processed/L2_LR_windwave/SWOT_L2_LR_SSH_2024-03-28_194356.037288.nc',
 '/home/yugao/SWOT_L2/data/processed/L2_LR_windwave/SWOT_L2_LR_SSH_2024-01-25_052812.553917.nc',
 '/home/yugao/SWOT_L2/data/processed/L2_LR_windwave/SWOT_L2_LR_SSH_2024-02-24_114513.678275.nc',
 '/home/yugao/SWOT_L2/data/processed/L2_LR_windwave/SWOT_L2_LR_SSH_2024-02-26_003652.553773.nc',
 '/home/yugao/SWOT_L2/data/processed/L2_LR_windwave/SWOT_L2_LR_SSH_2023-12-03_004452.746855.nc',
 '/home/yugao/SWOT_L2/data/processed/L2_LR_windwave/SWOT_L2_LR_SSH_2024-01-26_052843.716038.nc',
 '/home/yugao/SWOT_L2/data/pro

In [21]:
stride = 4  # Subsampling stride for vectors, increase stride for less density
# Define bad flags using the provided flag values
bad_flags = 33554432 | 536870912 | 1073741824 | 2147483648

# Minimum number of valid data points required to plot
min_valid_points = 60

In [22]:
for file_path in nc_files:
    with xr.open_dataset(file_path) as ds:
        if all(var in ds.variables for var in ['wind_speed_karin', 'wind_speed_karin_qual', 'wind_speed_model_u', 'wind_speed_model_v']):
            qual_flags = ds['wind_speed_karin_qual'].astype(int)
            good_mask = (qual_flags & bad_flags) == 0
            good_wind_speed = ds['wind_speed_karin'].where(good_mask, drop=True)
            u_component = ds['wind_speed_model_u'].where(good_mask, drop=True)
            v_component = ds['wind_speed_model_v'].where(good_mask, drop=True)
            latitude = ds['latitude'].where(good_mask, drop=True)
            longitude = ds['longitude'].where(good_mask, drop=True)

            # Extract start time from dataset attributes
            time_coverage_start = ds.attrs.get('time_coverage_start', 'No start time available')

            if good_wind_speed.count() >= min_valid_points:
                fig, axs = plt.subplots(1, 2, figsize=(15, 8), subplot_kw={'projection': ccrs.PlateCarree()})

                # Panel 1: Wind Speed
                ax1 = axs[0]
                ax1.set_extent(lolabox, crs=ccrs.PlateCarree())
                ax1.coastlines()
                ax1.gridlines(draw_labels=True, dms=False, x_inline=False, y_inline=False)
                scatter = ax1.scatter(longitude, latitude, c=good_wind_speed, cmap='inferno', s=30, edgecolor='none', transform=ccrs.PlateCarree())
                plt.colorbar(scatter, ax=ax1, orientation='horizontal', label='Wind Speed (m/s)')
                ax1.set_title('Wind Speed')  # Title for the first subplot

                # Panel 2: Wind Vectors with less density and wind speed overlay
                ax2 = axs[1]
                ax2.set_extent(lolabox, crs=ccrs.PlateCarree())
                ax2.coastlines()
                ax2.gridlines(draw_labels=True, dms=False, x_inline=False, y_inline=False)
                q = ax2.quiver(longitude[::stride], latitude[::stride], u_component[::stride], v_component[::stride], scale=160, transform=ccrs.PlateCarree(), color='black')
                scatter2 = ax2.scatter(longitude, latitude, c=good_wind_speed, cmap='inferno', s=30, edgecolor='none', alpha=0.5, transform=ccrs.PlateCarree())
                plt.colorbar(scatter2, ax=ax2, orientation='horizontal', label='Wind Speed (m/s)')
                ax2.set_title('Wind Speed and Direction')  # Title for the second subplot

                plt.suptitle(f'Wind Analysis - Start: {time_coverage_start}', fontsize=20)
                
                plt.savefig(os.path.join(img_dir, f'{os.path.basename(file_path)}_panels.png'), dpi=300)
                plt.close()
            else:
                print(f"Not enough valid data in {file_path} to plot.")
        else:
            print(f"Required data or quality flags not found in {file_path}")

/home/yugao/software/miniforge3/envs/swot/lib/python3.12/site-packages/xarray/core/duck_array_ops.py:215: RuntimeWarning: invalid value encountered in cast
  return data.astype(dtype, **kwargs)
/home/yugao/software/miniforge3/envs/swot/lib/python3.12/site-packages/xarray/core/duck_array_ops.py:215: RuntimeWarning: invalid value encountered in cast
  return data.astype(dtype, **kwargs)
/home/yugao/software/miniforge3/envs/swot/lib/python3.12/site-packages/xarray/core/duck_array_ops.py:215: RuntimeWarning: invalid value encountered in cast
  return data.astype(dtype, **kwargs)
/home/yugao/software/miniforge3/envs/swot/lib/python3.12/site-packages/xarray/core/duck_array_ops.py:215: RuntimeWarning: invalid value encountered in cast
  return data.astype(dtype, **kwargs)
/home/yugao/software/miniforge3/envs/swot/lib/python3.12/site-packages/xarray/core/duck_array_ops.py:215: RuntimeWarning: invalid value encountered in cast
  return data.astype(dtype, **kwargs)
/home/yugao/software/miniforge

Not enough valid data in /home/yugao/SWOT_L2/data/processed/L2_LR_windwave/SWOT_L2_LR_SSH_2024-03-28_194356.037288.nc to plot.


/home/yugao/software/miniforge3/envs/swot/lib/python3.12/site-packages/xarray/core/duck_array_ops.py:215: RuntimeWarning: invalid value encountered in cast
  return data.astype(dtype, **kwargs)
/home/yugao/software/miniforge3/envs/swot/lib/python3.12/site-packages/xarray/core/duck_array_ops.py:215: RuntimeWarning: invalid value encountered in cast
  return data.astype(dtype, **kwargs)
/home/yugao/software/miniforge3/envs/swot/lib/python3.12/site-packages/xarray/core/duck_array_ops.py:215: RuntimeWarning: invalid value encountered in cast
  return data.astype(dtype, **kwargs)
/home/yugao/software/miniforge3/envs/swot/lib/python3.12/site-packages/xarray/core/duck_array_ops.py:215: RuntimeWarning: invalid value encountered in cast
  return data.astype(dtype, **kwargs)
/home/yugao/software/miniforge3/envs/swot/lib/python3.12/site-packages/xarray/core/duck_array_ops.py:215: RuntimeWarning: invalid value encountered in cast
  return data.astype(dtype, **kwargs)
/home/yugao/software/miniforge

Not enough valid data in /home/yugao/SWOT_L2/data/processed/L2_LR_windwave/SWOT_L2_LR_SSH_2024-01-26_052843.716038.nc to plot.
Not enough valid data in /home/yugao/SWOT_L2/data/processed/L2_LR_windwave/SWOT_L2_LR_SSH_2024-02-16_021348.412724.nc to plot.


/home/yugao/software/miniforge3/envs/swot/lib/python3.12/site-packages/xarray/core/duck_array_ops.py:215: RuntimeWarning: invalid value encountered in cast
  return data.astype(dtype, **kwargs)
/home/yugao/software/miniforge3/envs/swot/lib/python3.12/site-packages/xarray/core/duck_array_ops.py:215: RuntimeWarning: invalid value encountered in cast
  return data.astype(dtype, **kwargs)
/home/yugao/software/miniforge3/envs/swot/lib/python3.12/site-packages/xarray/core/duck_array_ops.py:215: RuntimeWarning: invalid value encountered in cast
  return data.astype(dtype, **kwargs)
/home/yugao/software/miniforge3/envs/swot/lib/python3.12/site-packages/xarray/core/duck_array_ops.py:215: RuntimeWarning: invalid value encountered in cast
  return data.astype(dtype, **kwargs)
/home/yugao/software/miniforge3/envs/swot/lib/python3.12/site-packages/xarray/core/duck_array_ops.py:215: RuntimeWarning: invalid value encountered in cast
  return data.astype(dtype, **kwargs)
/home/yugao/software/miniforge

Not enough valid data in /home/yugao/SWOT_L2/data/processed/L2_LR_windwave/SWOT_L2_LR_SSH_2024-01-05_084337.968533.nc to plot.


/home/yugao/software/miniforge3/envs/swot/lib/python3.12/site-packages/xarray/core/duck_array_ops.py:215: RuntimeWarning: invalid value encountered in cast
  return data.astype(dtype, **kwargs)
/home/yugao/software/miniforge3/envs/swot/lib/python3.12/site-packages/xarray/core/duck_array_ops.py:215: RuntimeWarning: invalid value encountered in cast
  return data.astype(dtype, **kwargs)
/home/yugao/software/miniforge3/envs/swot/lib/python3.12/site-packages/xarray/core/duck_array_ops.py:215: RuntimeWarning: invalid value encountered in cast
  return data.astype(dtype, **kwargs)
/home/yugao/software/miniforge3/envs/swot/lib/python3.12/site-packages/xarray/core/duck_array_ops.py:215: RuntimeWarning: invalid value encountered in cast
  return data.astype(dtype, **kwargs)


Not enough valid data in /home/yugao/SWOT_L2/data/processed/L2_LR_windwave/SWOT_L2_LR_SSH_2023-11-24_151327.767694.nc to plot.


/home/yugao/software/miniforge3/envs/swot/lib/python3.12/site-packages/xarray/core/duck_array_ops.py:215: RuntimeWarning: invalid value encountered in cast
  return data.astype(dtype, **kwargs)
/home/yugao/software/miniforge3/envs/swot/lib/python3.12/site-packages/xarray/core/duck_array_ops.py:215: RuntimeWarning: invalid value encountered in cast
  return data.astype(dtype, **kwargs)
/home/yugao/software/miniforge3/envs/swot/lib/python3.12/site-packages/xarray/core/duck_array_ops.py:215: RuntimeWarning: invalid value encountered in cast
  return data.astype(dtype, **kwargs)
/home/yugao/software/miniforge3/envs/swot/lib/python3.12/site-packages/xarray/core/duck_array_ops.py:215: RuntimeWarning: invalid value encountered in cast
  return data.astype(dtype, **kwargs)


Not enough valid data in /home/yugao/SWOT_L2/data/processed/L2_LR_windwave/SWOT_L2_LR_SSH_2024-03-07_225852.209422.nc to plot.
Not enough valid data in /home/yugao/SWOT_L2/data/processed/L2_LR_windwave/SWOT_L2_LR_SSH_2023-12-15_115830.957875.nc to plot.


In [23]:
[v for v in ds.variables]

['time',
 'time_tai',
 'polarization_karin',
 'swh_karin',
 'swh_karin_qual',
 'swh_karin_uncert',
 'sig0_karin',
 'sig0_karin_qual',
 'sig0_karin_uncert',
 'sig0_karin_2',
 'sig0_karin_2_qual',
 'wind_speed_karin',
 'wind_speed_karin_qual',
 'wind_speed_karin_2',
 'wind_speed_karin_2_qual',
 'num_pt_avg',
 'swh_wind_speed_karin_source',
 'swh_wind_speed_karin_source_2',
 'swh_nadir_altimeter',
 'swh_model',
 'mean_wave_direction',
 'mean_wave_period_t02',
 'wind_speed_model_u',
 'wind_speed_model_v',
 'wind_speed_rad',
 'distance_to_coast',
 'heading_to_coast',
 'ancillary_surface_classification_flag',
 'dynamic_ice_flag',
 'rain_flag',
 'rad_surface_type_flag',
 'latitude',
 'longitude']

In [15]:
# ds.wind_speed_karin_2